# Download YOLOV5 and create our workspace 

In [ ]:

%cd /kaggle
!rm -rf tmp
!mkdir tmp
%cd tmp

In [ ]:
ls ../

In [ ]:
#Download YOLOv5

# !git clone https://github.com/ultralytics/yolov5  # clone repo

%cp -R /kaggle/input/yolov5-for-siim-covid19/yolov5 /kaggle/tmp
%cd yolov5
# %pip install -qr requirements.txt #install dependencies
%cp -R /kaggle/input/requirements-for-siim-covid19/requirements.txt  /kaggle/tmp/yolov5

%cd ../
import torch

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
# !ls ../kaggle/tmp
# !ls /kaggle/input/yolov5-for-siim-covid19/yolov5
# !ls /kaggle/input/../input/requirements-for-siim-covid19
# !ls /kaggle/input
# !ls tmp

In [ ]:
# !pip install -q --upgrade wandb
# !pip install --upgrade wanb==0.9.7
# #Login
# import wandb
# wandb.login()

# Import packages 

In [ ]:
# Necessary/extra dependencies. 
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import wandb
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

# Hyperparameters

In [ ]:
TRAIN_PATH = 'input/siim-covid19-resized-to-256px-jpg/train/'
IMG_SIZE = 256
BATCH_SIZE = 16
EPOCHS = 10

# Prepare Dataset

* Create train_validation split
* Create required `/dataset`
* Create `data.yaml` file needed to train the model
* Create bounding box coordinates in the required YOLO format

In [ ]:
# Everything is done from /kaggle directory
%cd /kaggle
#Load image level csv file
df = pd.read_csv('input/siim-covid19-detection/train_image_level.csv')

#Modify values in the id column
df['id'] = df.apply(lambda row:row.id.split('_')[0],axis=1)

# Add absolute path
df['path'] = df.apply(lambda row:TRAIN_PATH+row.id+'.jpg',axis=1)

#get image level labels
df['image_level'] = df.apply(lambda row:row.label.split(' ')[0], axis=1)
df.head(5)


In [ ]:
# Load meta.csv file
# Original dimensions are required to scale the bounding box coordinates appropriately.

meta_df = pd.read_csv('input/siim-covid19-resized-to-256px-jpg/meta.csv')
train_meta_df = meta_df.loc[meta_df.split=='train']
train_meta_df=train_meta_df.drop('split',axis=1)
train_meta_df.columns=['id','dim0','dim1']

train_meta_df.head()

In [ ]:
#merge train df
df = df.merge(train_meta_df,on='id',how='left')
df.head()

# Train & Validation set split

In [ ]:
train_df, valid_df = train_test_split(df, test_size=0.2, random_state = 710, stratify=df.image_level.values)


# train_df.assign(split = "train")
# valid_df.assign(split = "valid")

train_df.loc[:,"split"]='train'
valid_df.loc[:,"split"]='valid'

df = pd.concat([train_df, valid_df]).reset_index(drop=True)


In [ ]:
print(f'Size of dataset: {len(df)}, training images: {len(train_df)}. validation images: {len(valid_df)}')
df.head()

In [ ]:
os.makedirs('tmp/covid/images/train',exist_ok=True)
os.makedirs('tmp/covid/images/valid',exist_ok=True)


os.makedirs('tmp/covid/labels/train',exist_ok=True)
os.makedirs('tmp/covid/labels/valid',exist_ok=True)

!ls tmp/covid/images

In [ ]:
for i in tqdm(range(len(df))):
    row=df.loc[i]
    if row.split=='train':
        copyfile(row.path, f'tmp/covid/images/train/{row.id}.jpg')
    else:
        copyfile(row.path, f'tmp/covid/images/valid/{row.id}.jpg')

# Create `.YAML` file

In [ ]:
import yaml

data_yaml=dict(
    train='../covid/images/train',
    val='../covid/images/valid',
    nc=2,
    names=['none','opacity'])

#write yaml file
with open('tmp/yolov5/data/data.yaml','w') as outfile:
    yaml.dump(data_yaml,outfile,default_flow_style=True)

%cat tmp/yolov5/data/data.yaml

# Prepare Bounding Box Coordinated for YOLOv5

In [ ]:
# Get the raw rounding box by parsing the row value of the label column

def get_bbox(row):
    bboxes=[]
    bbox=[]
    for i,l in enumerate(row.label.split(' ')):
        if (i%6==0)|(i%6==1):
            continue
        bbox.append(float(l))
        if (i%6==5):
            bboxes.append(bbox)
            bbox=[]
    return bboxes

#Scale the bounding boxes according to the size of the resized image

def scale_bbox(row, bboxes):
    # Get scaling factor
    
    scale_x = IMG_SIZE/row.dim1
    scale_y= IMG_SIZE/row.dim0
    
    scaled_bboxes=[]
    
    for bbox in bboxes:
        x = int(np.round(bbox[0]*scale_x,4))
        y = int(np.round(bbox[1]*scale_y,4))
        x1=int(np.round(bbox[2]*scale_x,4))
        y1=int(np.round(bbox[2]*scale_y,4))
    
        scaled_bboxes.append([x,y,x1,y1])
        
        
    return scaled_bboxes

# convert the bounding boxes into YOLO format

def get_yolo_format_bbox(img_w,img_h,bboxes):
    yolo_boxes=[]
    for bbox in bboxes:
        w=bbox[2]-bbox[0]
        h=bbox[3]-bbox[1]
        xc=bbox[0]+int(np.round(w/2)) # mid point of height and width
        yc=bbox[1]+int(np.round(h/2))
        #normalization?
        yolo_boxes.append([xc/img_w,yc/img_h,w/img_w,h/img_h])
    
    return yolo_boxes

In [ ]:
# prepare the txt files for the bounding box

for i in tqdm(range(len(df))):
    row = df.loc[i]
    
    #get image id
    img_id=row.id
    split=row.split
    label=row.image_level
    
    if row.split=='train':
        file_name=f'tmp/covid/labels/train/{row.id}.txt'
    
    else:
        file_name=f'tmp/covid/labels/valid/{row.id}.txt'
        
        
    if label=='opacity':
        
        #Get bboxes
        bboxes = get_bbox(row)
        # Scale bounding boxes
        scale_bboxes = scale_bbox(row,bboxes)
        
        # Format for YOLOv5
        yolo_bboxes=get_yolo_format_bbox(IMG_SIZE,IMG_SIZE,scale_bboxes)
        
        with open(file_name,'w') as f:
            for bbox in yolo_bboxes:
                bbox=[1]+bbox
                bbox=[str(i) for i in bbox]
                bbox = ' '.join(bbox)
                f.write(bbox)
                f.write('\n')
        
        
        

In [ ]:
%cd tmp/yolov5/

In [ ]:
# !python train.py --img {IMG_SIZE} \
#                  --batch {BATCH_SIZE} \
#                  --epochs {EPOCHS} \
#                  --data data.yaml \
#                  --weights yolov5s.pt \
#                  --save_period 1\
#                  --project kaggle-siim-covid

In [ ]:
# cd /kaggle/tmp/yolov5/kaggle-siim-covid/exp/weights/
%ls

In [ ]:
# # import wandb
# iimport wandb
# run = wandb.init()
# artifact = run.use_artifact('andernzhu/kaggle-siim-covid/run_3a0jaej5_model:v9', type='model')
# artifact_dir = artifact.download()

In [ ]:
ls /kaggle/tmp/yolov5

In [ ]:
TEST_PATH = '/kaggle/input/siim-covid19-resized-to-256px-jpg/test/' # absolute path

In [ ]:
MODEL_PATH = '/kaggle/input/best-model/317a481ae22f8b127f462ec81d721bea'

In [ ]:
cd /kaggle/tmp/yolov5

In [ ]:
!python detect.py --weights {MODEL_PATH} \
                  --source {TEST_PATH} \
                  --img {IMG_SIZE} \
                  --conf 0.281 \
                  --iou-thres 0.5 \
                  --max-det 3 \
                  --save-txt \
                  --save-conf

In [ ]:
PRED_PATH = 'runs/detect/exp/labels'
# !ls {PRED_PATH}


In [ ]:
# Visualize predicted coordinates.
%cat runs/detect/exp2/labels/ffb8115a304c.txt

In [ ]:
prediction_files = os.listdir(PRED_PATH)
print('Number of test images predicted as opaque: ', len(prediction_files))

# Generate the submission `.csv` file

In [ ]:
# The submisison requires xmin, ymin, xmax, ymax format. 
# YOLOv5 returns x_center, y_center, width, height
def correct_bbox_format(bboxes):
    correct_bboxes = []
    for b in bboxes:
        xc, yc = int(np.round(b[0]*IMG_SIZE)), int(np.round(b[1]*IMG_SIZE))
        w, h = int(np.round(b[2]*IMG_SIZE)), int(np.round(b[3]*IMG_SIZE))

        xmin = xc - int(np.round(w/2))
        xmax = xc + int(np.round(w/2))
        ymin = yc - int(np.round(h/2))
        ymax = yc + int(np.round(h/2))
        
        correct_bboxes.append([xmin, xmax, ymin, ymax])
        
    return correct_bboxes

# Read the txt file generated by YOLOv5 during inference and extract 
# confidence and bounding box coordinates.
def get_conf_bboxes(file_path):
    confidence = []
    bboxes = []
    with open(file_path, 'r') as file:
        for line in file:
            preds = line.strip('\n').split(' ')
            preds = list(map(float, preds))
            confidence.append(preds[-1])
            bboxes.append(preds[1:-1])
    return confidence, bboxes
        

In [ ]:
# Read the submission file

sub_df=pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
sub_df.tail()

In [ ]:
# Prediction loop for submission
predictions = []

for i in tqdm(range(len(sub_df))):
    row = sub_df.loc[i]
    id_name = row.id.split('_')[0]
    id_level = row.id.split('_')[-1]
    
    if id_level == 'study':
        # do study-level classification
        predictions.append("Negative 1 0 0 1 1") # dummy prediction
        
    elif id_level == 'image':
        # we can do image-level classification here.
        # also we can rely on the object detector's classification head.
        # for this example submisison we will use YOLO's classification head. 
        # since we already ran the inference we know which test images belong to opacity.
        if f'{id_name}.txt' in prediction_files:
            # opacity label
            confidence, bboxes = get_conf_bboxes(f'{PRED_PATH}/{id_name}.txt')
            bboxes = correct_bbox_format(bboxes)
            pred_string = ''
            for j, conf in enumerate(confidence):
                pred_string += f'opacity {conf} ' + ' '.join(map(str, bboxes[j])) + ' '
            predictions.append(pred_string[:-1]) 
        else:
            predictions.append("None 1 0 0 1 1") 

In [ ]:
sub_df['PredictionString'] = predictions
sub_df.to_csv('submission.csv', index=False)
sub_df.tail()

In [ ]:
%mv submission.csv /kaggle/working
# %mv /kaggle/tmp/yolov5/requirements.txt /kaggle/working
%ls /kaggle/working 